In [1]:
# coding=utf-8
from __future__ import print_function

import json
import os
import pickle
import sys
from itertools import chain

import numpy as np

from asdl.asdl import ASDLGrammar
from asdl.hypothesis import Hypothesis
from datasets.wikisql.lib.common import detokenize
from datasets.wikisql.lib.dbengine import DBEngine
from datasets.wikisql.lib.query import Query
from asdl.lang.sql.sql_transition_system import SqlTransitionSystem, sql_query_to_asdl_ast, asdl_ast_to_sql_query
from datasets.wikisql.utils import my_detokenize, find_sub_sequence
from asdl.transition_system import GenTokenAction
from components.action_info import ActionInfo
from components.vocab import VocabEntry, Vocab
from model.wikisql.dataset import WikiSqlExample, WikiSqlTable, TableColumn

In [2]:
from datasets.wikisql.dataset import load_dataset
from datasets.wikisql.dataset import get_action_infos

In [3]:
engine = DBEngine('tranx.0.2.0/data/wikisql/train.db')
data_file = 'dsl_parser/my_wikisql/wikisql_train.json'
grammar = ASDLGrammar.from_text(open('asdl/lang/sql/sql_asdl.txt').read())

transition_system = SqlTransitionSystem(grammar)

In [4]:
query = Query(1, 5)
query

SELECT AVG col1 FROM table

In [5]:
query.to_dict()

{'sel': 1, 'agg': 5, 'conds': []}

In [6]:
asdl_ast = sql_query_to_asdl_ast(query, grammar)
asdl_ast.sanity_check()
actions = transition_system.get_actions(asdl_ast)

In [7]:
actions

[ApplyRule[stmt -> Select(agg_op? agg, column_idx col_idx, cond_expr* conditions)],
 ApplyRule[agg_op -> Avg()],
 SelectColumnAction[id=1],
 Reduce]

In [8]:
query_reconstr = asdl_ast_to_sql_query(asdl_ast)

In [9]:
query_reconstr

SELECT AVG col1 FROM table